In [3]:
from neo4j import GraphDatabase

In [56]:
driver = GraphDatabase.driver('bolt://localhost:7687')

In [36]:
# 4

name = "Stan Wokulski"
title = "Lalka"

def create_actor(driver, name):
    with driver.session() as session:
        result = session.run('CREATE (a:Actor{name: $name}) RETURN a', name=name)
        print(list(result))
        
def create_movie(driver, title):
    with driver.session() as session:
        result = session.run('CREATE (m:Movie{title: $title}) RETURN m', title=title)
        print(list(result))
   
     
def create_relation(driver, name, title):
    with driver.session() as session:
        result = session.run(
                '''
                MATCH (a:Actor{name: $name}), (m:Movie{title: $title})
                CREATE (a) -[:ACTS_IN]-> (m)
                ''', name=name, title=title)
        print(list(result))
    

              
create_actor(driver, name)
create_movie(driver, title)
create_relation(driver, name, title)

[<Record m=<Node id=219931 labels={'Movie'} properties={'title': 'Lalka'}>>]
[]


In [40]:
# 5

name = "Stan Wokulski"
birthplace = 'Warsaw'
birthdate = '09-07-1832'

def set_actor_information(driver, name, birthplace, birthdate):
    with driver.session() as session:
        result = session.run(
                '''
                MATCH (a:Actor{name: $name})
                SET a.birthplace = $birthplace
                SET a.birthdate = $birthdate
                ''', name=name, birthplace=birthplace, birthdate=birthdate)
        print(list(result))
           
set_actor_information(driver, name, birthplace, birthdate)

[]


In [48]:
# 6

def count_query(driver):
    with driver.session() as session:
        result = session.run(
            '''
            MATCH (a:Actor) -[r:ACTS_IN]-> (m:Movie) 
            WITH a, length( collect(r) ) as count 
            WHERE count >= 6 
            RETURN a
            ''')
    

count_query(driver)

In [44]:
# 7

def average_query(driver):
    with driver.session() as session:
        result = session.run(
                  '''
                  MATCH (a:Actor) -[r:ACTS_IN]-> (m:Movie) 
                  WITH a, length( collect(r) ) as count 
                  WHERE count >= 7 
                  RETURN AVG(count)
                  ''')
        print(list(result)) 

average_query(driver)

[<Record AVG(count)=12.775387263339091>]


In [47]:
# 8

def actor_director_query(driver):
    with driver.session() as session:
        result = session.run(
                    '''
                    MATCH () <-[d:DIRECTED]- (a:Actor) -[p:ACTS_IN]-> () 
                    WITH a, count(d) AS c1, count(p) AS c2 
                    WHERE c1 >= 5 AND c2 >= 1
                    RETURN a
                    ORDER BY c1
                    ''')

actor_director_query(driver)

In [53]:
# 9

login = 'maheshksp'

def rating_query(driver, login):
    with driver.session() as session:
        result = session.run(
                '''
                MATCH (u:User{login: $login}) -[:FRIEND]-> (f:User) -[r:RATED] -> (m:Movie) 
                WHERE r.stars >= 3 
                RETURN f, m.title, r.stars
                ''', login=login)
    
rating_query(driver, login)

In [64]:
#10

actor1 = "Emma Watson"
actor2 = "Daniel Radcliffe"

def path_query(driver, actor1, actor2):
    with driver.session() as session:
        result = session.run(
                    '''
                    MATCH path = (a1:Actor {name: $actor1}) -[*1..6]- (a2:Actor {name: $actor2})
                    RETURN DISTINCT [n IN nodes(path) WHERE NOT "Movie" IN LABELS(n) | n.name]
                    ''', actor1=actor1, actor2=actor2)
        

path_query(driver, actor1, actor2)